## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [23]:
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice


def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [124]:
import networkx as nx

def lower_bound_tsp(vertex_coordinates: List[Tuple[int,int]]) -> float:
    n = len(vertex_coordinates)
    pi = [0] * n
    T = nx.Graph()
    edges = []
    alpha = 40
    ans = 0
    for it in range(30):
        G = nx.Graph()
        for i in range(0, n):
            for j in range(i + 1, n):
                G.add_edge(i, j, weight=euclidean_distance(vertex_coordinates[i], vertex_coordinates[j]) + pi[i] + pi[j])
        T = nx.minimum_spanning_tree(G)
        mi1 = 1000000007
        mi2 = 1000000007
        for j in range(1, n):
            leng = euclidean_distance(vertex_coordinates[0], vertex_coordinates[j]) + pi[0] + pi[j]
            if (leng < mi1):
                mi2 = mi1
                mi1 = leng
                a = j
            elif (leng < mi2):
                mi2 = leng
                b = j
        T.add_edge(0, a, weight=euclidean_distance(vertex_coordinates[0], vertex_coordinates[a]) + pi[0] + pi[a])
        T.add_edge(0, b, weight=euclidean_distance(vertex_coordinates[0], vertex_coordinates[b]) + pi[0] + pi[b])
        edges = T.edges(data=False)
        deg = [0] * n
        for i in range(len(edges)):
            deg[edges[i][0]] += 1
            deg[edges[i][1]] += 1
        sum = 0
        edges = T.edges(data=True)
        for i in range(len(edges)):
            sum += edges[i][2]['weight'] - pi[edges[i][0]] - pi[edges[i][1]]
        ans = max(ans, sum)
        for i in range(0, n):
             pi[i] += (deg[i] - 2) * alpha / (it + 1)
    
    return ans

In [118]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        bound = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))

In [125]:
run_all()

Instance d198.tsp… done in 1.8e+01 seconds with lower bound 15417
Instance d493.tsp… done in 9.8e+01 seconds with lower bound 37128
Instance d657.tsp… done in 1.7e+02 seconds with lower bound 52114
Instance d2103.tsp… done in 1.9e+03 seconds with lower bound 82747
Instance pr107.tsp… done in 3.9 seconds with lower bound 37868
Instance pr152.tsp… done in 7.8 seconds with lower bound 61694
Instance pr439.tsp… done in 7.1e+01 seconds with lower bound 96113


Верхние оценки из решения задачи 3-1:
Solving instance d198.tsp… done with tour length 18620 using NN and tour length 18052 using NI
Solving instance d493.tsp… done with tour length 43646 using NN and with tour length 41576 using NI
Solving instance d657.tsp… done with tour length 62176 using NN and with tour length 60195 using NI
Solving instance d2103.tsp… done with tour length 87468 using NN and with tour length 86027 using NI
Solving instance pr107.tsp… done with tour length 46678 using NN and with tour length 53211 using NI
Solving instance pr152.tsp… done with tour length 85702 using NN and with tour length 86914 using NI
Solving instance pr439.tsp… done with tour length 131282 using NN and with tour length 132780 using NI

## Выводы
Алгоритмы NN и NI решают задачу TSP достаточно эффективно, так как, как мы можем видеть, их результаты не более чем в 1.4 раза больше, чем оценка Гельда—Карпа.